In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
from splinter import Browser

In [2]:
# Scrape the  NASA Mars News Site 
# url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
url = "https://mars.nasa.gov/news/"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
response = browser.html
soup = BeautifulSoup(response, 'html.parser')

In [3]:
# print(soup.prettify())

In [4]:
news_t = soup.find('li', class_='slide').find('div',class_="content_title").find('a').text
news_p = soup.find('li', class_='slide').find('div', class_="rollover_description_inner").text

In [5]:
news_t

'My Culture, My Voice'

In [6]:
news_p

'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.'

In [7]:
# Scrape JPL Featured Space Image
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [8]:
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [9]:
browser.find_link_by_partial_text('FULL IMAGE').first.click()

C:\Users\asiha\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [10]:
browser.find_link_by_partial_text('more info').first.click()

In [11]:
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

In [12]:
# print(image_soup.prettify())

In [13]:
f_image = image_soup.select_one("figure.lede a img").get("src")
f_image

'/spaceimages/images/largesize/PIA17011_hires.jpg'

In [14]:
featured_image_url = 'https://www.jpl.nasa.gov' +f_image
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17011_hires.jpg'

In [15]:
# Scrape Mars Facts webpage

table_url = 'https://space-facts.com/mars/'
tables = pd.read_html(table_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [16]:
df = tables[0]
df.columns = ['Index', 'Info']
df.set_index('Index', inplace=True)
df

,Info
Index,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
df.index.name = None
df.columns=['']
df

,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
# Scrape USGS Astrogeology site
hem_list=[]
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hem_response = requests.get(hem_url)
hem_soup = BeautifulSoup(hem_response.text, 'html.parser')

In [19]:
# print(hem_soup.prettify())

In [20]:

for item in hem_soup.find_all('h3'):
    hem_list.append(item.text)



In [21]:
hem_browser = Browser('chrome', **executable_path, headless=False)
h_image_url = [] 
for item in hem_list:
    hem_browser.visit(hem_url)
    hem_browser.find_link_by_partial_text(item).click()
    hem_browser.find_link_by_partial_text('Open').click()
    hem_html = hem_browser.html
    hem_soup = BeautifulSoup(hem_html, 'html.parser')
    h_image = hem_soup.select_one("div.wide-image-wrapper img.wide-image").get("src")
    h_image_url.append({"title":item, "img_url":'https://astrogeology.usgs.gov'+ h_image})

    
    
    

C:\Users\asiha\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [22]:
h_image_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]